In [10]:
import pandas as pd
# Performance Evaluation and Support
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [17]:
!wget https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!wget https://raw.githubusercontent.com/google-research/bert/master/modeling.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/optimization.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/run_classifier.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/tokenization.py 

'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
# Load the data into the dataframe 
data = pd.read_csv('..\\data\\data.csv')
data.shape


(1650, 11)

In [3]:
data.columns

Index(['Unnamed: 0', 'Title', 'Score', 'ID', 'URL', 'num_comments',
       'created_on', 'Body', 'Original', 'Flair', 'Comments'],
      dtype='object')

In [4]:
# Data Shuffling
data[:] = data.sample(frac=1).values
data.drop(['Unnamed: 0'], inplace=True, axis=1)
data.head()

,Title,Score,ID,URL,num_comments,created_on,Body,Original,Flair,Comments
0,39 fresh coronavirus cases reported in Kerala ...,44,fpx3mf,https://www.livemint.com/news/india/39-fresh-c...,12,1.585345e+09,NaN,False,Policy/Economy,They say they are most educated state in India...
1,Hiranandani Hospital charging Rs 6400 for a Co...,698,g2xdk7,https://imgur.com/a/gKIAH6I,80,1.587138e+09,NaN,False,Non-Political,I don't know if I have lost touch with India i...
2,63 Indian Billionaires Have More Money Than Th...,1330,er6bom,https://www.ndtv.com/india-news/63-indian-bill...,179,1.579515e+09,NaN,False,Business/Finance,Hmm. There's income inequality and then there'...
3,What's your favorite truly wireless earbuds as...,26,f0lom5,https://www.reddit.com/r/india/comments/f0lom5...,27,1.581160e+09,I'm looking to buy a pair of tws earbuds but t...,False,Science/Technology,I'm happy with - onemore stylish. It costs aro...
4,Seven IITs drop out of participation in Times ...,169,g2y4cq,https://www.moneycontrol.com/news/business/eco...,24,1.587142e+09,NaN,False,Science/Technology,New IITs (Ropar) beat the old IITs\n\nOld iit ...


In [5]:
# Printing the data info to have a look at the null values and data types
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1650 entries, 0 to 1649
Data columns (total 10 columns):
Title           1650 non-null object
Score           1650 non-null int64
ID              1650 non-null object
URL             1650 non-null object
num_comments    1650 non-null int64
created_on      1650 non-null float64
Body            635 non-null object
Original        1650 non-null bool
Flair           1650 non-null object
Comments        1557 non-null object
dtypes: bool(1), float64(1), int64(2), object(6)
memory usage: 117.7+ KB


In [6]:
# List of relevant features (MOVE LATER)
features = ['Flair', 'URL', 'Title', 'Comments', 'Body']

# Collecting the flair and ids
data = data[features]
data.head()

,Flair,URL,Title,Comments,Body
0,Policy/Economy,https://www.livemint.com/news/india/39-fresh-c...,39 fresh coronavirus cases reported in Kerala ...,They say they are most educated state in India...,NaN
1,Non-Political,https://imgur.com/a/gKIAH6I,Hiranandani Hospital charging Rs 6400 for a Co...,I don't know if I have lost touch with India i...,NaN
2,Business/Finance,https://www.ndtv.com/india-news/63-indian-bill...,63 Indian Billionaires Have More Money Than Th...,Hmm. There's income inequality and then there'...,NaN
3,Science/Technology,https://www.reddit.com/r/india/comments/f0lom5...,What's your favorite truly wireless earbuds as...,I'm happy with - onemore stylish. It costs aro...,I'm looking to buy a pair of tws earbuds but t...
4,Science/Technology,https://www.moneycontrol.com/news/business/eco...,Seven IITs drop out of participation in Times ...,New IITs (Ropar) beat the old IITs\n\nOld iit ...,NaN


In [7]:
# Assigning and individual id to each flair
data['id'] = data['Flair'].factorize()[0]
flair_category = data[['Flair', 'id']].drop_duplicates().sort_values('id')
flair_category

,Flair,id
0,Policy/Economy,0
1,Non-Political,1
2,Business/Finance,2
3,Science/Technology,3
5,AMA,4
6,AskIndia,5
8,Politics,6
11,Photography,7
16,Sports,8
17,Food,9


In [8]:
data['Combine'] = data['Title'] # Create a column combined
count = 0
for i in range(len(data)):
    if type(data.loc[i]['Body']) != float:
        data['Combine'][i] = data['Combine'][i] + ' ' + data['Body'][i]

    if type(data.loc[i]['Comments']) != float:
        data['Combine'][i] = data['Combine'][i] + ' ' + data['Comments'][i]

data.head(20)

c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,Flair,URL,Title,Comments,Body,id,Combine
0,Policy/Economy,https://www.livemint.com/news/india/39-fresh-c...,39 fresh coronavirus cases reported in Kerala ...,They say they are most educated state in India...,NaN,0,39 fresh coronavirus cases reported in Kerala ...
1,Non-Political,https://imgur.com/a/gKIAH6I,Hiranandani Hospital charging Rs 6400 for a Co...,I don't know if I have lost touch with India i...,NaN,1,Hiranandani Hospital charging Rs 6400 for a Co...
2,Business/Finance,https://www.ndtv.com/india-news/63-indian-bill...,63 Indian Billionaires Have More Money Than Th...,Hmm. There's income inequality and then there'...,NaN,2,63 Indian Billionaires Have More Money Than Th...
3,Science/Technology,https://www.reddit.com/r/india/comments/f0lom5...,What's your favorite truly wireless earbuds as...,I'm happy with - onemore stylish. It costs aro...,I'm looking to buy a pair of tws earbuds but t...,3,What's your favorite truly wireless earbuds as...
4,Science/Technology,https://www.moneycontrol.com/news/business/eco...,Seven IITs drop out of participation in Times ...,New IITs (Ropar) beat the old IITs\n\nOld iit ...,NaN,3,Seven IITs drop out of participation in Times ...
5,AMA,https://www.reddit.com/r/india/comments/23nxii...,"IAMA guy who quit his job, sold his house and ...",Weirdest thing that you've ever eaten?\n\nAlso...,Hello! \n\nI am Vahishta. In Dec 2012 I quit m...,4,"IAMA guy who quit his job, sold his house and ..."
6,AskIndia,https://www.reddit.com/r/india/comments/fxq3g9...,"Urgent, please help",Please tell your dad to write letters to the c...,"My dad is stuck in Madhya Pradesh (Shahdole), ...",5,"Urgent, please help My dad is stuck in Madhya ..."
7,Science/Technology,https://www.reddit.com/r/india/comments/ftn4l0...,Why are the net banking sites absolute and utt...,दLit Engineers,It's a pity that these applications used by mi...,3,Why are the net banking sites absolute and utt...
8,Politics,https://www.reddit.com/r/india/comments/futac9...,Pitting a community against a political party ...,Are you Muslim OP?,First of all let me start by saying it was stu...,6,Pitting a community against a political party ...
9,Politics,https://indianexpress.com/article/india/corona...,BJP chief Nadda cautions party leaders: Don’t ...,Pot. Kettle. Black.,NaN,6,BJP chief Nadda cautions party leaders: Don’t ...


In [11]:
# Splitting 20% of the data into train test split
X_train, X_test, y_train, y_test = train_test_split(data['Combine'], data['Flair'], test_size=0.15, random_state=42)
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(1402,) (1402,) (248,) (248,)


In [16]:
train_InputExamples = X_train.apply(lambda x: bert.run_classifier.InputExample(guid=None,
                                                                   text_a = x['Combine'], 
                                                                   text_b = None, 
                                                                   label = x['id']))
val_InputExamples = X_test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x['Combine'], 
                                                                   text_b = None, 
                                                                   label = x['id']))


NameError: name 'bert' is not defined

In [ ]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128

# Convert our train and validation features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

val_features = bert.run_classifier.convert_examples_to_features(val_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)